# 2. Page Rank

In [5]:
class ShortestPath:
    def __init__(self, nodes, edges, initial_node):
        self.nodes = SparkContext.getOrCreate().parallelize(nodes).map(lambda node: (node, float('inf')))
        self.edges = SparkContext.getOrCreate().parallelize(edges).map(lambda edge: (edge[0], (edge[1], edge[2]))).groupByKey().mapValues(list)
        self.nodes = self.nodes.map(lambda node: (node[0], 0) if node[0] == initial_node else node)

    @staticmethod
    def prepare_messages(node):
        node_id, (cost, neighbors) = node
        for neighbor, edge_cost in neighbors:
            yield (neighbor, cost + edge_cost)

    @staticmethod
    def exchange_messages(messages):
        return messages.reduceByKey(lambda a, b: min(a, b))

    def calculate(self, max_iterations=100):
        nodes = self.nodes
        edges = self.edges
        for i in range(max_iterations):
            prev_nodes = nodes
            join_nodes_edges = nodes.join(edges)
            messages = join_nodes_edges.flatMap(self.prepare_messages)
            final_messages = self.exchange_messages(messages)
            nodes = final_messages.leftOuterJoin(prev_nodes).map(lambda x: (x[0], x[1][0] if x[1][1] is None or x[1][0] < x[1][1] else x[1][1]))

            if nodes.join(prev_nodes).map(lambda x: x[1][0] == x[1][1]).reduce(lambda a, b: a and b):
                print(f"La ejecución terminó por convergencia en {i+1} iteraciones.")
                break

        else:
            print(f"La ejecución terminó porque se alcanzó el máximo de {max_iterations} iteraciones.")

        self.nodes = nodes
        return self.nodes.collect()

In [7]:
sc = SparkContext.getOrCreate()

nodes = [1, 2, 3, 4]
edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]

shortest_path = ShortestPath(nodes, edges, 1)
result = shortest_path.calculate(max_iterations=10)

for node in result:
    print(f"El costo mínimo para llegar al nodo {node[0]} es: {node[1]}")

La ejecución terminó por convergencia en 3 iteraciones.
El costo mínimo para llegar al nodo 2 es: 10
El costo mínimo para llegar al nodo 3 es: 13
El costo mínimo para llegar al nodo 4 es: 34
